In [6]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import pytz
import json
import ta_formulas as ta
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import boto3
import ast

s3 = boto3.client('s3')

In [9]:
def pull_closed_orders_s3(path, bucket):
    dfs = []
    keys = s3.list_objects(Bucket=bucket,Prefix=f"{path}")["Contents"]
    for object in keys:
        key = object['Key']
        dataset = s3.get_object(Bucket=bucket,Key=f"{key}")
        df = pd.read_csv(dataset.get("Body"))
        df.drop(columns=['Unnamed: 0'],inplace=True)
        if len(df) > 0:
            dfs.append(df)
    full_df = pd.concat(dfs)
    full_df.reset_index(drop=True)
    return full_df

In [10]:
path = 'closed_orders/PROD_VAL/CDBFC_1D'
bucket = 'inv-alerts-trading-data'

df = pull_closed_orders_s3(path, bucket)

df.head()

,open_order_id,closing_order_id,option_symbol,position_id
0,10831957,10832538,UBER240322C00079000,UBER-CDBFC1D-2024-03-18T15-05
1,10831962,10832541,UBER240322C00080000,UBER-CDBFC1D-2024-03-18T15-05
0,10831952,10833538,IWM240322C00209000,IWM-CDBFC1D-2024-03-18T15-05
0,10835525,10846786,SNOW240322C00170000,SNOW-CDBFC1D-2024-03-18T17-07
1,10833362,10846791,SNOW240322C00170000,SNOW-CDBFC1D-2024-03-18T16-06


In [11]:
orders_list = df['open_order_id'].to_list()

In [21]:
client = boto3.client('dynamodb')

def extract_values_from_dict(d):
    return list(d.values())[0] if isinstance(d, dict) else d

def get_open_trades_by_orderid(order_id_list):
    partitions = break_array_into_partitions(order_id_list)
    df_list = []
    error_list = []
    for partition in partitions:
        try:
            response = client.batch_get_item(
                RequestItems={
                    'icarus-closed-orders-table-inv': {
                        'Keys': [{'order_id': {'S': str(id)}} for id in partition]
                    }
                }
            )
            
            items = response['Responses']['icarus-closed-orders-table-inv']
            result_df = pd.DataFrame(items)
            result_df = result_df.map(extract_values_from_dict)
            df_list.append(result_df)
        except:
            error_list.append(partition)
    full_df = pd.concat(df_list)
    return full_df, error_list

def break_array_into_partitions(arr):
    """
    Breaks an array into equal partitions of less than 100 items each.

    Args:
        arr (list): The input array.

    Returns:
        list: A list of partitions, where each partition is a list of items.
    """
    if len(arr) <= 100:
        # If the array has 100 or fewer items, return it as-is
        return [arr]
    else:
        # Calculate the number of partitions needed
        num_partitions = (len(arr) + 99) // 100  # Round up to the nearest integer

        # Calculate the size of each partition
        partition_size = len(arr) // num_partitions

        # Initialize the starting index and ending index for each partition
        start = 0
        end = partition_size

        # Create the partitions
        partitions = []
        for i in range(num_partitions):
            # If it's the last partition, include any remaining items
            if i == num_partitions - 1:
                partitions.append(arr[start:])
            else:
                partitions.append(arr[start:end])

            # Update the starting and ending index for the next partition
            start = end
            end += partition_size

        return partitions

In [22]:
orders_df, errors = get_open_trades_by_orderid(orders_list)
print(len(orders_list)
print(len(errors))

/var/folders/dy/0184f__n1gndgrhsxps0zl4c0000gn/T/ipykernel_49264/4147048398.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result_df = result_df.applymap(extract_values_from_dict)


[[11573106, 11573109, 11585299, 11575765, 11585286, 11585288, 11585266, 11585268, 11585264, 11585285, 11586987, 11586985, 11586986, 11585250, 11576771, 11586997, 11586992, 11586995, 11586994, 11585252, 11586991, 11586984, 11585300, 11585272, 11585273, 11585270, 11585302, 11588139, 11586990, 11588137, 11585277, 11588138, 11586988, 11585283, 11585281, 11585275, 11585279, 11588135, 11588141, 11589108, 11585262, 11588130, 11585257, 11585259, 11573115, 11588127, 11588133, 11589124, 11586983, 11585304, 11589122, 11585247, 11589111, 11589119, 11588144, 11588142, 11589120, 11589109, 11589104, 11589117, 11574793, 11589113, 11585306, 11573116, 11589106, 11590022, 11590005, 11590024, 11589128, 11589126, 11590007, 11590008, 11589129, 11590026, 11588147, 11585308, 11590031, 11590011, 11590012, 11585311, 11590010, 11590033, 11588148, 11588150, 11585313, 11590036, 11590028, 11585294, 11585290, 11585292, 11591138, 11591135, 11591139, 11591141, 11591133, 11590020, 11591154, 11590017, 11591128, 11591125